<a href="https://colab.research.google.com/github/acesur/Machine-Learning-/blob/main/utils_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

def extract_abcd_features_robust(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return None

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        _, thresh_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        thresh_adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                cv2.THRESH_BINARY_INV, 11, 2)

        for thresh in [thresh_otsu, thresh_adaptive]:
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours:
                continue

            largest_contour = max(contours, key=cv2.contourArea)
            contour_area = cv2.contourArea(largest_contour)
            if contour_area < 100:
                continue

            mask = np.zeros_like(gray)
            cv2.drawContours(mask, [largest_contour], 0, 255, -1)

            h, w = mask.shape
            flipped_mask = cv2.flip(mask, 1)
            asymmetry = np.sum(np.abs(mask - flipped_mask)) / (h * w * 255)

            perimeter = cv2.arcLength(largest_contour, True)
            border_irregularity = perimeter**2 / (4 * np.pi * contour_area) if contour_area > 0 else 0

            r, g, b = cv2.split(img)
            mask_bool = mask > 0

            color_variance_r = np.var(r[mask_bool]) if np.sum(mask_bool) > 0 else 0
            color_variance_g = np.var(g[mask_bool]) if np.sum(mask_bool) > 0 else 0
            color_variance_b = np.var(b[mask_bool]) if np.sum(mask_bool) > 0 else 0

            x, y, w, h = cv2.boundingRect(largest_contour)
            diameter = max(w, h)

            return {
                'asymmetry': float(asymmetry),
                'border_irregularity': float(border_irregularity),
                'color_variance_r': float(color_variance_r),
                'color_variance_g': float(color_variance_g),
                'color_variance_b': float(color_variance_b),
                'diameter': float(diameter)
            }

        return None

    except Exception as e:
        print(f"Error in feature extraction: {str(e)}")
        return None
